<a href="https://colab.research.google.com/github/lalesafarzade/Recommendation_system_Project/blob/lale/Notebooks/5.best_movies_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
import requests
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
%matplotlib inline
#from configs import api_key

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
movie_df=pd.read_csv('/content/drive/MyDrive/movies.csv')
link_df=pd.read_csv('/content/drive/MyDrive/links.csv')
rating_df=pd.read_csv('/content/drive/MyDrive/ratings.csv')
json_metadata=pd.read_json('/content/drive/MyDrive/metadata_updated.json', lines=True)

In [4]:
movie=pd.merge(link_df, movie_df, on="movieId")
movie.drop(['tmdbId'],axis=1,inplace=True)
df1=json_metadata[['imdbId','directedBy','starring','avgRating']]
mvielense_df = pd.merge(movie, df1, on=["imdbId"],how='inner')
mvielense_df.head(2)


,movieId,imdbId,title,genres,directedBy,starring,avgRating
0,1,114709,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",3.89146
1,2,113497,Jumanji (1995),Adventure|Children|Fantasy,Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",3.26605


In [5]:
df = pd.merge(rating_df,mvielense_df,on='movieId')
df.head()

,userId,movieId,rating,timestamp,imdbId,title,genres,directedBy,starring,avgRating
0,1,296,5.0,1147880044,110912,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,Quentin Tarantino,"John Travolta, Samuel L. Jackson, Tim Roth, Am...",4.1876
1,3,296,5.0,1439474476,110912,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,Quentin Tarantino,"John Travolta, Samuel L. Jackson, Tim Roth, Am...",4.1876
2,4,296,4.0,1573938898,110912,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,Quentin Tarantino,"John Travolta, Samuel L. Jackson, Tim Roth, Am...",4.1876
3,5,296,4.0,830786155,110912,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,Quentin Tarantino,"John Travolta, Samuel L. Jackson, Tim Roth, Am...",4.1876
4,7,296,4.0,835444730,110912,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,Quentin Tarantino,"John Travolta, Samuel L. Jackson, Tim Roth, Am...",4.1876


#### Making a Ratings dataframe with average rating and number of ratings:

In [8]:
ratings = pd.DataFrame(df.groupby('title').mean())[['rating','avgRating']]
ratings['num of ratings'] = pd.DataFrame(df.groupby('title')['rating'].count())
ratings.head(2)

,rating,avgRating,num of ratings
title,,,
"""BLOW THE NIGHT!"" Let's Spend the Night Together (1983)",3.000000,3.00000,1
"""Great Performances"" Cats (1998)",2.896648,2.87205,179


In [10]:
#finding best movies by IMDB formula
mean_vote_report= df['avgRating'].mean()
minimum_votes= ratings['num of ratings'].quantile(0.9)
def weighted_rating(x, m=minimum_votes, C=mean_vote_report):
    v = x['num of ratings']
    R = x['avgRating']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

q_movies =ratings.copy().loc[ratings['num of ratings']>=minimum_votes]
q_movies['score'] = q_movies.apply(weighted_rating, axis= 1)
q_movies.sort_values(by= 'score', ascending= False, inplace= True)
movie_150=q_movies.head(150).reset_index()
movie_150= movie_150.assign(year = lambda x: x['title'].str[-5:-1])
movie_150


,title,rating,avgRating,num of ratings,score,year
0,"Shawshank Redemption, The (1994)",4.413576,4.41985,81482,4.415375,1994
1,"Godfather, The (1972)",4.324336,4.33078,52498,4.324550,1972
2,Planet Earth (2006),4.464797,4.45460,1747,4.278340,2006
3,"Usual Suspects, The (1995)",4.284353,4.27727,55366,4.271758,1995
4,"Godfather: Part II, The (1974)",4.261759,4.26899,34188,4.260203,1974
...,...,...,...,...,...,...
145,Django Unchained (2012),4.004882,4.02187,20687,4.012309,2012
146,Creature Comforts (1989),4.107460,4.09128,2480,4.011638,1989
147,Stalker (1979),4.079925,4.08462,2665,4.010659,1979
148,It's a Wonderful Life (1946),4.032859,4.02307,15384,4.010268,1946


In [20]:
df2=json_metadata[['title','imdbId','directedBy','starring','avgRating']]
df2[df2['title'].isin(movie_150['title'].to_list())]
best_movies = pd.merge(movie_150,df2[['title','directedBy','starring']],on='title')
best_movies

,title,rating,avgRating,num of ratings,score,year,directedBy,starring
0,"Shawshank Redemption, The (1994)",4.413576,4.41985,81482,4.415375,1994,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi..."
1,"Godfather, The (1972)",4.324336,4.33078,52498,4.324550,1972,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ..."
2,Planet Earth (2006),4.464797,4.45460,1747,4.278340,2006,Alastair Fothergill,David Attenborough
3,"Usual Suspects, The (1995)",4.284353,4.27727,55366,4.271758,1995,Bryan Singer,"Stephen Baldwin, Gabriel Byrne, Benicio Del To..."
4,"Godfather: Part II, The (1974)",4.261759,4.26899,34188,4.260203,1974,Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert..."
...,...,...,...,...,...,...,...,...
145,Django Unchained (2012),4.004882,4.02187,20687,4.012309,2012,Quentin Tarantino,"Jamie Foxx, Christoph Waltz, Leonardo DiCaprio..."
146,Creature Comforts (1989),4.107460,4.09128,2480,4.011638,1989,Nick Park,Julie Sedgewick
147,Stalker (1979),4.079925,4.08462,2665,4.010659,1979,Andrei Tarkovsky,"Nikolai Grinko, Aleksandr Kajdanovsky, Anatoli..."
148,It's a Wonderful Life (1946),4.032859,4.02307,15384,4.010268,1946,Frank Capra,"James Stewart,Donna Reed,Lionel Barrymore,Thom..."


In [23]:

from google.colab import files
best_movies.to_csv('best_movies.csv',index=False)
files.download('best_movies.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
#For Website
best_movies=best_movies.set_index("title")
html=best_movies.to_html()
# write html to file
text_file = open("../templates/best_movies.html", "w")
text_file.write(html)
text_file.close()